In [15]:
import tensorflow as tf
import keras
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation, BatchNormalization, Input
from keras.layers.embeddings import Embedding

In [88]:
max_words = 305718
batch_size = 32
epochs = 10
test_size = 0.3
def create_conv_model():
    classifier = Sequential()
    #1.Multiple convolution and max pooling
    classifier.add(Conv1D(filters=8, kernel_size=11, activation="relu"))
    classifier.add(MaxPooling1D(strides=4))
    classifier.add(BatchNormalization())
    classifier.add(Conv1D(filters=16, kernel_size=11, activation='relu'))
    classifier.add(MaxPooling1D(strides=4))
    classifier.add(BatchNormalization())
    classifier.add(Conv1D(filters=32, kernel_size=11, activation='relu'))
    classifier.add(MaxPooling1D(strides=4))
    classifier.add(BatchNormalization())

    #2.Flattening
    classifier.add(Flatten())

    #3.Full Connection
    classifier.add(Dropout(0.5))
    classifier.add(Dense(64, activation='relu'))
    classifier.add(Dropout(0.25))
    classifier.add(Dense(64, activation='relu'))
    classifier.add(Dense(1, activation='sigmoid'))

    #Configure the learning process
    classifier.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return classifier

In [178]:
import pandas as pd
training_set = pd.read_csv('water_data.csv',index_col=None, header=0)
training_set.fillna(0, inplace=True)
training_set.head()

,Unnamed: 0,STATE,DISTRICT,TEH_NAME,BLOCK_NAME,LAT,LON,SITE_NAME,SITE_TYPE,WLCODE,YEAR_OBS,MONSOON,POSTMONSOONKHARIF,POSTMONSOONRABI,PREMONSOON
0,0,WB,Bankura,Jaypur,Bishnupur,23.061111,87.404167,Aima,Dug Well,W10158,2017,.7,0,3.1,2.03
1,1,WB,Bankura,Bishnupur,Bishnupur,23.063333,87.330000,Bishnupur-1,Tube Well,W10121,2017,3,0,3.1,0
2,2,WB,Bankura,Indas,Indas,23.130278,87.648889,Indas1,Dug Well,W10150,2017,0,0,2.92,0
3,3,WB,Bankura,Taldangra,Simlapal,23.033333,87.063889,Ghalkunda,Dug Well,W10215,2017,2.38,0,4.38,6.44
4,4,WB,Bankura,Ranibandh,Ranibandh,22.863889,86.797222,Ranibandh-2,Dug Well,W21189,2017,4.6,0,8.3,0


In [179]:
training_set.describe()

,Unnamed: 0,LAT,LON,YEAR_OBS
count,305718.000000,305718.000000,305718.000000,305718.000000
mean,445.666745,20.885937,78.944542,2007.272732
std,375.676164,5.881965,4.741621,6.584195
min,0.000000,8.083333,68.533333,1994.000000
25%,152.000000,16.833333,75.781944,2002.000000
50%,345.000000,21.637500,77.775000,2008.000000
75%,646.000000,25.241667,81.850000,2013.000000
max,2143.000000,34.616944,96.133333,2017.000000


In [180]:
unlabelled_in_all = training_set[(training_set['STATE']!=1) & (training_set['DISTRICT']!=1) & (training_set['TEH_NAME']!=1) & 
                            (training_set['BLOCK_NAME']!=1) & (training_set['LAT']!=1) & (training_set['LON']!=1) & (training_set['SITE_NAME']!=1) & (training_set['SITE_TYPE']!=1) & (training_set['WLCODE']!=1)]
print('Percentage of unlabelled comments is ', len(unlabelled_in_all)/len(training_set)*100)

Percentage of unlabelled comments is  100.0


In [181]:
import csv
import numpy as np
from sklearn.model_selection import train_test_split
#Importing training set
X = training_set.loc[:,'MONSOON':'PREMONSOON'].values
# Xmax = training_set.loc[:,'PREMONSOON'].max
# X = training_set.loc[:,'MONSOON':'PREMONSOON'].divide(Xmax)
# print(X)
y = training_set.loc[:,'STATE': 'YEAR_OBS'].values
X_train, y_train, X_test, y_test = train_test_split(X,y, test_size=0.3)
m = create_conv_model()
m.fit(X_train, y_train, batch_size=batch_size, validation_data=(X_test,y_test), epochs=epochs, verbose=1)
# m.train_on_batch(x_batch, y_batch)
score = m.evaluate(X_test, y_test)
# from skmultilearn.adapt import MLkNN

# classifier = MLkNN(k=20)

# # train
# classifier.fit(X_train, y_train)

# # predict
# predictions = classifier.predict(X_test)

# score = accuracy_score(y_test,predictions)
print('Test score:', score[0])
print('Test accuracy:', score[1])

ValueError: Input 0 is incompatible with layer conv1d_50: expected ndim=3, found ndim=2

In [72]:
nn = Sequential()
nn.add(Dense(10, activation="relu", input_shape=(10,)))
nn.add(Dense(5))

In [73]:
import math

def softmax(z):
    z_exp = [math.exp(i) for i in z]
    sum_z_exp = sum(z_exp)
    return [i / sum_z_exp for i in z_exp]


In [77]:
z = [1.0, 2.0, 3.0, 4.0, 1.0]
softmax(z)

[0.031062774127550943,
 0.0844373744524495,
 0.22952458061688552,
 0.623912496675563,
 0.031062774127550943]

In [176]:
def sigmoid(z):
    return [1 / (1 + math.exp(-n)) for n in z]

In [79]:
z = [-1.0, 5.0, -0.5, 5.0, -0.5]
sigmoid(z)

[0.2689414213699951,
 0.9933071490757153,
 0.3775406687981454,
 0.9933071490757153,
 0.3775406687981454]

In [80]:
nn = Sequential()
nn.add(Dense(10, activation="relu", input_shape=(10,)))
nn.add(Dense(5, activation="sigmoid"))

In [81]:
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])